# Triage (Projeto desenvolvido para o Case Prático da empresa AutoU)
## Nesse arquivo: Especificação do processo de treinamento do modelo principal

## Descrição do case prático da AutoU

O objetivo central do case é **automatizar a leitura** e **sugerir classificações e respostas automáticas** de acordo com o teor de cada email recebido.

Existem duas categorias de classificação:
- Emails **Produtivos**: Emails que requerem uma ação ou resposta específica (ex.: solicitações de suporte técnico, atualização sobre casos em aberto, dúvidas sobre o sistema); e
- Emails **Improdutivos**: Emails que não necessitam de uma ação imediata (ex.: mensagens de felicitações, agradecimentos).

## Descrição da solução

Para atigir esses objetivos, proponho o **Triage**, um sistema especialmente criado para esse case prático com foco na classificação de emails e resposta automática. Em resumo, o modelo principal do Triage foi treinado através de um
processo de *fine-tuning* no modelo popularmente conhecido [DistilBERT base multilingual](https://huggingface.co/distilbert/distilbert-base-multilingual-cased) para especializa-lo na tarefa principal de classíficação de e-mails.

Para verificar o modelo treinado, ele foi exportado no formato Hugging Face Transformes e está disponível aqui: https://huggingface.co/gabubits/triage-portuguese

Para testar a aplicação, ela está deployada e hospedada no Hugging Face Spaces (https://huggingface.co/spaces/gabubits/triage-demo-email-classifier).

Observação: O treinamento do modelo foi realizado em GPU.

In [ ]:
# Instalação de bibliotecas principais

!pip install evaluate -q # Para o cálculo métricas de avaliação do modelo.
!pip install transformers
!pip install datasets
!pip install tqdm
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install torch

In [53]:
# Biblioteca principal para trabalhar com modelos pré-treinados
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers import DataCollatorWithPadding

# Para adaptar o modelo ao formato compatível com as especificações do Hugging Face
from datasets import Dataset, DatasetDict

# Usada para exibir barras de progresso, útil durante operações demoradas como o tokenization.
from tqdm import tqdm

import evaluate

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [34]:
# Carregamento dos dados para treinamento do modelo
# O dataset foi populado com e-mails sintéticos gerados pelo Gemini.
# O dataset tem três colunas: category (Produtivo ou Improdutivo), subject (assunto do e-mail) e content (conteúdo do e-mail).

df = pd.read_csv("dataset_portugues.csv", sep=";")
df.head()

,category,subject,content
0,Produtivo,URGENTE: Erro de login no módulo Gestão de Atlas.,"Olá, O sistema de login do módulo Gestão de At..."
1,Produtivo,Solicitação de orçamento - Geração de Cristal.,"Prezado(a), Gostaria de solicitar um orçamento..."
2,Produtivo,Dúvida sobre o procedimento de Crono-Iniciação.,"Olá, No novo procedimento de Crono-Iniciação, ..."
3,Produtivo,Feedback pendente para aprovação do design Aur...,"Prezado(a), O prazo para aprovação do design A..."
4,Produtivo,Agendamento de Reunião de Start-up Cósmico.,"Olá Equipe, Proponho três horários para nossa ..."


In [35]:
# Preparação inicial dos dados

# category é renomeada para labels, padronizando o nome da coluna alvo.
df['labels'] = df['category']

# subject e content são concatenadas em uma nova coluna chamada text, que servirá como a entrada de texto principal para o modelo. Um prefixo ('SUBJECT: ' e ' BODY: ') é adicionado para distinguir as partes do e-mail.
df['text'] = 'SUBJECT: ' + df['subject'] + ' BODY: ' + df['content']

# As colunas originais (`category`, `subject`, `content`) são removidas do DataFrame, pois seus dados agora estão representados nas novas colunas `labels` e `text`.
df = df.drop(columns=['category', 'subject', 'content'])
df.head()

,labels,text
0,Produtivo,SUBJECT: URGENTE: Erro de login no módulo Gest...
1,Produtivo,SUBJECT: Solicitação de orçamento - Geração de...
2,Produtivo,SUBJECT: Dúvida sobre o procedimento de Crono-...
3,Produtivo,SUBJECT: Feedback pendente para aprovação do d...
4,Produtivo,SUBJECT: Agendamento de Reunião de Start-up Có...


In [36]:
# Como evidenciado abaixo, a distribuição entre as classes é relativamente equilibrada.

category_count = df['labels'].value_counts()
category_count

,count
labels,
Improdutivo,200
Produtivo,191


In [37]:
# Conversão das labels categórias textuais para números.
# Os modelos geralmente requerem entradas numéricas para o treinamento.

df['labels'] = df['labels'].astype('category').cat.codes
df.head()

,labels,text
0,1,SUBJECT: URGENTE: Erro de login no módulo Gest...
1,1,SUBJECT: Solicitação de orçamento - Geração de...
2,1,SUBJECT: Dúvida sobre o procedimento de Crono-...
3,1,SUBJECT: Feedback pendente para aprovação do d...
4,1,SUBJECT: Agendamento de Reunião de Start-up Có...


In [38]:
# Separação dos dados de texto (`text`) e as labels (`labels`) em variáveis distintas
# Esta separação é feita em preparação para a divisão do dataset em conjuntos de treino, validação e teste.

data_texts = df['text']

data_labels = df['labels']

In [39]:
# Divisão do dataset em conjuntos de treinamento e validação
# test_size=0.3 especifica que 30% dos dados serão alocados para o conjunto de validação/teste, e os 70% restantes para o conjunto de treinamento.
# random_state=42 garante a reprodutibilidade da divisão, ou seja, a cada execução com o mesmo `random_state`, a divisão será a mesma.
# stratify=data_labels é utilizado para garantir que a proporção das classes (Produtivo/Improdutivo) nos conjuntos de treinamento e validação seja a mesma.

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.3, random_state = 42, stratify=data_labels )

In [40]:
# Constrói um DatasetDict com os dados de treinamento e de validação.
# O uso de DatasetDict e Dataset é eficiente a integração com o Trainer do Hugging Face.

train_dict = {'text': train_texts, 'labels': train_labels}
val_dict = {'text': val_texts, 'labels': val_labels}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': val_dataset
})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 273
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 118
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 118
    })
})

In [43]:
# Carregamento do tokenizador pré-treinado associado ao modelo 'distilbert-base-multilingual-cased'.

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [42]:
# Carregamento do modelo pré-treinado 'distilbert-base-multilingual-cased'
# AutoModelForSequenceClassification é uma classe conveniente que carrega o modelo ('distilbert-base-multilingual-cased' neste caso) e adiciona uma cabeça de classificação linear no topo.
# num_labels=2 especifica que a cabeça de classificação deve ter duas unidades de saída, correspondendo às duas classes do nosso problema de classificação de e-mails (Produtivo e 'mprodutivo).
# id2label e label2id são dicionários que mapeiam os IDs numéricos das classes (0 e 1) para seus nomes textuais e vice-versa.

id2label = {0: "Improdutivo", 1: "Produtivo"}
label2id = {"Improdutivo": 0, "Produtivo": 1}

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels = 2, id2label = id2label, label2id = label2id)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Este bloco de código configura quais partes do modelo pré-treinado terão seus pesos atualizados durante o processo de fine-tuning.

for name, param in model.base_model.named_parameters():
    param.requires_grad = False

for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [45]:
# Etapa de pre-processamento

# Aplicação da função preprocess_function ao Dataset. Esta função utiliza o tokenizador carregado para converter o texto bruto de cada e-mail em um formato que o modelo DistilBERT pode entender.

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [46]:
# Pre-processamento de todos os datasets definidos no DatasetDict criado.

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

Map:   0%|          | 0/118 [00:00<?, ? examples/s]

Map:   0%|          | 0/118 [00:00<?, ? examples/s]

In [47]:
# DataCollator agrupa múltiplas amostras processadas do dataset em batches para o treinamento ou avaliação do modelo.
# O padding é aplicado apenas até o comprimento da sequência mais longa dentro de cada batch individual.
# O benefício dessa técnica é minimiza\ a quantidade de tokens de preenchimento desnecessários, especialmente quando as sequências no dataset variam em comprimento (como nesse caso, que são textos de emails).

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [48]:
# Definição da função de cálculo de métricas para o processo de treinamento.

# A acurácia mede a proporção de previsões corretas.
accuracy = evaluate.load("accuracy")

# O AUC mede a capacidade do modelo de distinguir entre as classes.
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # Faz predições
    predictions, labels = eval_pred

    # Aplica softmax para as probabilidades
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # Usa as probabilidades para cálculo do AUC
    positive_class_probs = probabilities[:, 1]
    # Calcula AUC
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # Verifica a categoria mais provável
    predicted_classes = np.argmax(predictions, axis=1)
    # Calcula a acurácia
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

In [49]:
# Definição dos argumentos do processo de treinamento do modelo usando o Trainer da biblioteca transformers.
# Estes argumentos controlam diversos aspectos do treinamento, otimização e avaliação.

lr = 2e-4 # Número comum para fine-tuning em modelos de transformadores
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="triage-model-training-tests",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

In [50]:
# Inicialização do Trainer, classe principal na biblioteca transformers para orquestrar o processo de treinamento e avaliação de modelos.
# O Trainer abstrai muitos dos detalhes do ciclo de treinamento, como otimização, agendamento da taxa de aprendizado, cálculo de perdas e gerenciamento de dispositivos.

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Principal bloco: o processo de treinamento (fine-tuning) do modelo.
# Ao chamar o método trainer.train(), o Trainer executa o loop de treinamento de acordo com os TrainingArguments configurados.

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.668500,0.598433,0.678000,0.943000
2,0.533400,0.463545,0.881000,0.942000
3,0.422500,0.375214,0.881000,0.950000
4,0.373100,0.340290,0.881000,0.957000
5,0.337500,0.291616,0.890000,0.969000
6,0.280300,0.269509,0.890000,0.973000
7,0.255800,0.252432,0.898000,0.974000
8,0.259200,0.245954,0.907000,0.976000
9,0.245000,0.238303,0.898000,0.977000
10,0.236600,0.235451,0.890000,0.977000


TrainOutput(global_step=350, training_loss=0.3611845397949219, metrics={'train_runtime': 242.8952, 'train_samples_per_second': 11.239, 'train_steps_per_second': 1.441, 'total_flos': 68702715705684.0, 'train_loss': 0.3611845397949219, 'epoch': 10.0})

In [ ]:
# Etapa final de avaliação do modelo treinado no conjunto de validação
# Como exposto abaixo, o melhor modelo atingiu uma acurácia de 89% e AUC em torno de 97%.

predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.89), 'AUC': np.float64(0.977)}


In [ ]:
# Métricas de avaliação detalhadas do modelo.

trainer.evaluate()

{'eval_loss': 0.23545145988464355,
 'eval_Accuracy': 0.89,
 'eval_AUC': 0.977,
 'eval_runtime': 0.442,
 'eval_samples_per_second': 266.974,
 'eval_steps_per_second': 33.937,
 'epoch': 10.0}

In [ ]:
# Salva o modelo DistilBERT que foi fine-tuned e o tokenizador correspondente em um diretório local.
# Esses arquivos estão disponíveis no github para execução local, além de estarem disponíveis na página
# principal do modelo no Hugging Faces (https://huggingface.co/gabubits/triage-portuguese)
save_directory = "/triage-model-portuguese"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/triage-model-portuguese/tokenizer_config.json',
 '/triage-model-portuguese/special_tokens_map.json',
 '/triage-model-portuguese/vocab.txt',
 '/triage-model-portuguese/added_tokens.json',
 '/triage-model-portuguese/tokenizer.json')

In [ ]:
# Carregamento do modelo treinado

tokenizer_fine_tuned = AutoTokenizer.from_pretrained(save_directory)

model_fine_tuned = AutoModelForSequenceClassification.from_pretrained(save_directory)

In [ ]:
# Teste do modelo treinado

import torch

predict_input = tokenizer_fine_tuned.encode(
    "A copa do escritório recebeu uma nova decoração para deixar o ambiente mais agradável. Esperamos que gostem.",
    truncation = True,
    padding = True,
    return_tensors = 'pt'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = torch.argmax(output, dim = 1).item()

print(f"The predicted label ID is: {id2label[prediction_value]}")

The predicted label ID is: Improdutivo


Todos os arquivos da pasta `/triage-model-portuguese` foram upados na plataforma Hugging Face no link https://huggingface.co/gabubits/triage-portuguese